In [ ]:
# https://machinelearningmastery.com/crash-course-recurrent-neural-networks-deep-learning/
# https://machinelearningmastery.com/memory-in-a-long-short-term-memory-network/

In [10]:
from keras.layers import LSTM, Dense
from keras.models import Sequential
import pandas as pd

In [14]:
# binary encode an input pattern, return a list of binary vectors
def encode(pattern, n_unique):
    encoded = list()
    for value in pattern:
        row = [0.0 for x in range(n_unique)]
        row[value] = 1.0
        encoded.append(row)
    return encoded

# create input/output pairs of encoded vectors, returns X, y
def to_xy_pairs(encoded):
    X,y = list(),list()
    for i in range(1, len(encoded)):
        X.append(encoded[i-1])
        y.append(encoded[i])
    return X, y

# convert sequence to x/y pairs ready for use with an LSTM
def to_lstm_dataset(sequence, n_unique):
    # one hot encode
    encoded = encode(sequence, n_unique)
    # convert to in/out patterns
    X,y = to_xy_pairs(encoded)
    # convert to LSTM friendly format
    dfX, dfy = pd.DataFrame(X), pd.DataFrame(y)
    lstmX = dfX.values
    #lstmX = lstmX.reshape(lstmX.shape[0], 1, lstmX.shape[1])
    lstmX = lstmX.reshape(lstmX.shape[0], lstmX.shape[1], 1)
    lstmY = dfy.values
    return lstmX, lstmY

seq1 = [3, 0, 1, 2, 3]
encoded = encode(seq1, 5)
for vector in encoded:
    print(vector)

seq1 = [3, 0, 1, 2, 3]
encoded = encode(seq1, 5)
X, y = to_xy_pairs(encoded)
for i in range(len(X)):
    print(X[i], y[i])


df = pd.DataFrame(X)
print(df)
values = df.values
array = values.reshape(4, 1, 5)
#array = values.reshape(4, 5, 1)

seq1 = [3, 0, 1, 2, 3]
seq2 = [4, 0, 1, 2, 4]
n_unique = len(set(seq1 + seq2))
 
seq1X, seq1Y = to_lstm_dataset(seq1, n_unique)
seq2X, seq2Y = to_lstm_dataset(seq2, n_unique)

model = Sequential()
#model.add(LSTM(20, batch_input_shape=(1, 1, 5), stateful=True))
model.add(LSTM(20, batch_input_shape=(1, 5, 1)))
model.add(Dense(5, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')


# train LSTM
# because this model is stateful, the timesteps can be size 1 and the sequence will be remembered
# interestingly, Brownlee uses the sequence as different features rather than timesteps
# I have added code to flip this for comparison, I suspect the model will do better with more timesteps rather than features
for i in range(250):
    model.fit(seq1X, seq1Y, epochs=1, batch_size=1, verbose=1, shuffle=False)
    model.reset_states()
    model.fit(seq2X, seq2Y, epochs=1, batch_size=1, verbose=0, shuffle=False)
    model.reset_states()
    
result = model.predict_classes(seq1X, batch_size=1, verbose=0)

[0.0, 0.0, 0.0, 1.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 1.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 1.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 1.0, 0.0]
[0.0, 0.0, 0.0, 1.0, 0.0] [1.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0] [0.0, 1.0, 0.0, 0.0, 0.0]
[0.0, 1.0, 0.0, 0.0, 0.0] [0.0, 0.0, 1.0, 0.0, 0.0]
[0.0, 0.0, 1.0, 0.0, 0.0] [0.0, 0.0, 0.0, 1.0, 0.0]
     0    1    2    3    4
0  0.0  0.0  0.0  1.0  0.0
1  1.0  0.0  0.0  0.0  0.0
2  0.0  1.0  0.0  0.0  0.0
3  0.0  0.0  1.0  0.0  0.0
Epoch 1/1
4/4 [==============================] - 1s 251ms/step - loss: 0.6920
Epoch 1/1
4/4 [==============================] - 0s 5ms/step - loss: 0.6845
Epoch 1/1
4/4 [==============================] - 0s 4ms/step - loss: 0.6775
Epoch 1/1
4/4 [==============================] - 0s 5ms/step - loss: 0.6702
Epoch 1/1
4/4 [==============================] - 0s 4ms/step - loss: 0.6623
Epoch 1/1
4/4 [==============================] - 0s 4ms/step - loss: 0.6535
Epoch 1/1
4/4 [==============================] - 0s 6ms/step - loss: 

4/4 [==============================] - 0s 4ms/step - loss: 0.3565
Epoch 1/1
4/4 [==============================] - 0s 3ms/step - loss: 0.3539
Epoch 1/1
4/4 [==============================] - 0s 2ms/step - loss: 0.3513
Epoch 1/1
4/4 [==============================] - 0s 4ms/step - loss: 0.3488
Epoch 1/1
4/4 [==============================] - 0s 3ms/step - loss: 0.3462
Epoch 1/1
4/4 [==============================] - ETA: 0s - loss: 0.286 - 0s 4ms/step - loss: 0.3435
Epoch 1/1
4/4 [==============================] - 0s 4ms/step - loss: 0.3410
Epoch 1/1
4/4 [==============================] - 0s 3ms/step - loss: 0.3384
Epoch 1/1
4/4 [==============================] - 0s 5ms/step - loss: 0.3359
Epoch 1/1
4/4 [==============================] - 0s 3ms/step - loss: 0.3335
Epoch 1/1
4/4 [==============================] - 0s 5ms/step - loss: 0.3311
Epoch 1/1
4/4 [==============================] - 0s 3ms/step - loss: 0.3287
Epoch 1/1
4/4 [==============================] - 0s 3ms/step - loss: 0.326

4/4 [==============================] - 0s 4ms/step - loss: 0.1493
Epoch 1/1
4/4 [==============================] - 0s 4ms/step - loss: 0.1484
Epoch 1/1
4/4 [==============================] - 0s 3ms/step - loss: 0.1475
Epoch 1/1
4/4 [==============================] - 0s 4ms/step - loss: 0.1465
Epoch 1/1
4/4 [==============================] - 0s 4ms/step - loss: 0.1455
Epoch 1/1
4/4 [==============================] - 0s 4ms/step - loss: 0.1447
Epoch 1/1
4/4 [==============================] - 0s 4ms/step - loss: 0.1436
Epoch 1/1
4/4 [==============================] - 0s 3ms/step - loss: 0.1428
Epoch 1/1
4/4 [==============================] - 0s 4ms/step - loss: 0.1418
Epoch 1/1
4/4 [==============================] - 0s 5ms/step - loss: 0.1410
Epoch 1/1
4/4 [==============================] - 0s 4ms/step - loss: 0.1402
Epoch 1/1
4/4 [==============================] - 0s 3ms/step - loss: 0.1394
Epoch 1/1
4/4 [==============================] - 0s 4ms/step - loss: 0.1384
Epoch 1/1
4/4 [=======